In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from scipy.optimize import minimize

In [ ]:
class CPM(object):
    """
    """
    def __init__(self, fits_file, remove_bad="True"):
        self.file_name = fits_file.split("/")[-1]  # Should I really keep this here?
        with fits.open(fits_file, mode="readonly") as hdulist:
            self.time = hdulist[1].data["TIME"]
            self.im_fluxes = hdulist[1].data["FLUX"]  # Shape is (1282, 64, 64)
            self.im_errors = hdulist[1].data["FLUX_ERR"]  # Shape is (1282, 64, 64)
            self.quality = hdulist[1].data["QUALITY"]
            
        # If remove_bad is set to True, we'll remove the values with a nonzero entry in the quality array
        if remove_bad == True:
            print("Removing bad values by using the TESS provided \"QUALITY\" array")
            b = (self.quality == 0)  # The zero value entries for quality array are the "good" values
            self.time = self.time[b]
            self.im_fluxes = self.im_fluxes[b]
            self.im_errors = self.im_errors[b]
            
        # Calculate the vandermode matrix to add polynomial components to model
        self.scaled_centered_time = ((self.time - (self.time.max() + self.time.min())/2) 
                                    / (self.time.max() - self.time.min()))
        self.v_matrix = np.vander(self.scaled_centered_time, N=4, increasing=True)
        
        self.target_row = None
        self.target_col = None
        self.target_fluxes = None
        self.target_errors = None
        self.target_median = None
        self.rescaled_target_fluxes = None
        self.rescaled_target_errors = None
        self.target_pixel_mask = None
        
        self.excluded_pixels_mask = None
        
        # We're going to precompute the pixel lightcurve medians since it's used to set the predictor pixels
        # but never has to be recomputed
        self.pixel_medians = np.median(self.im_fluxes, axis=0)
        self.flattened_pixel_medians = self.pixel_medians.reshape(self.im_fluxes[0].shape[0]**2)
        
        # We'll precompute the rescaled values for the fluxes (F* = F/M - 1)
        self.rescaled_im_fluxes = (self.im_fluxes/self.pixel_medians) - 1
        
        self.method_predictor_pixels = None
        self.num_predictor_pixels = None
        self.predictor_pixels_locations = None
        self.predictor_pixels_mask = None
        self.predictor_pixels_fluxes = None
        self.rescaled_predictor_pixels_fluxes = None
        
        self.fit = None
        self.regularization = None
        self.lsq_params = None
        self.cpm_params = None
        self.poly_params = None
        self.cpm_prediction = None
        self.poly_prediction = None
        self.prediction = None
        self.im_predicted_fluxes = None
        self.im_diff = None
        
        self.is_target_set = False
        self.is_exclusion_set = False
        self.are_predictors_set = False
        self.trained = False
        
    def set_target(self, target_row, target_col):
        self.target_row = target_row
        self.target_col = target_col
        self.target_fluxes = self.im_fluxes[:, target_row, target_col]  # target pixel lightcurve
        self.target_errors = self.im_errors[:, target_row, target_col]  # target pixel errors
        self.target_median = np.median(self.target_fluxes)
        self.rescaled_target_fluxes = self.rescaled_im_fluxes[:, target_row, target_col]
        self.rescaled_target_errors = self.target_errors / self.target_median
        
        target_pixel = np.zeros(self.im_fluxes[0].shape)
        target_pixel[target_row, target_col] = 1
        self.target_pixel_mask = np.ma.masked_where(target_pixel == 0, target_pixel)  # mask to see target
        
        self.is_target_set = True
        
    def set_exclusion(self, exclusion, method="cross"):
        if self.is_target_set == False:
            print("Please set the target pixel to predict using the set_target() method.")
            return
        
        r = self.target_row  # just to reduce verbosity for this function
        c = self.target_col
        exc = exclusion
        im_side_length = self.im_fluxes.shape[1]  # for convenience
        
        excluded_pixels = np.zeros(self.im_fluxes[0].shape)
        if method == "cross":
            excluded_pixels[max(0,r-exc) : min(r+exc+1, im_side_length), :] = 1
            excluded_pixels[:, max(0,c-exc) : min(c+exc+1, im_side_length)] = 1
            
        if method == "row_exclude":
            excluded_pixels[max(0,r-exc) : min(r+exc+1, im_side_length), :] = 1
        
        if method == "col_exclude":
            excluded_pixels[:, max(0,c-exc) : min(c+exc+1, im_side_length)] = 1
        
        if method == "closest":
            excluded_pixels[max(0,r-exc) : min(r+exc+1, im_side_length), 
                            max(0,c-exc) : min(c+exc+1, im_side_length)] = 1
        
        self.excluded_pixels_mask = np.ma.masked_where(excluded_pixels == 0, excluded_pixels)  # excluded pixel is "valid" and therefore False
        self.is_exclusion_set = True
    
    def set_predictor_pixels(self, num_predictor_pixels, method="similar_brightness", seed=None):
        if seed != None:
            np.random.seed(seed=seed)
        
        if (self.is_target_set == False) or (self.is_exclusion_set == False):
            print("Please set the target pixel and exclusion.")
            return 
            
        self.method_predictor_pixels = method
        self.num_predictor_pixels = num_predictor_pixels
        im_side_length = self.im_fluxes.shape[1]  # for convenience (I need column size to make this work)
        
        # I'm going to do this in 1D by assinging individual pixels a single index instead of two.
        coordinate_idx = np.arange(im_side_length**2)
        possible_idx = coordinate_idx[self.excluded_pixels_mask.mask.ravel()]
        
        if method == "random":
            chosen_idx = np.random.choice(possible_idx, size=num_predictor_pixels, replace=False)
        
        # Since it turns out that calculating the median of the pixels across time is somewhat expensive,
        # we're going to optimize the following by doing the median calculation at the time of instantiating
        # the CPM instead of calculating it each time the function is used.
#         if method == "similar_brightness":
#             target_median = np.median(self.target_fluxes)  # Median value of target lightcurve
#             pixel_medians = np.median(self.im_fluxes, axis=0)  # 64x64 matrix of median values of lightcurves
            
#             flattened_pixel_medians = pixel_medians.reshape(im_side_length**2)  # 4096 length array
#             possible_pixel_medians = flattened_pixel_medians[self.excluded_pixels_mask.mask.ravel()]
            
#             diff = (np.abs(possible_pixel_medians - target_median))
#             chosen_idx = possible_idx[np.argsort(diff)[0:self.num_predictor_pixels]]

        # Optimized version of above
        if method == "similar_brightness":
#             target_median = np.median(self.target_fluxes)
            
            possible_pixel_medians = self.flattened_pixel_medians[self.excluded_pixels_mask.mask.ravel()]
#             diff = (np.abs(possible_pixel_medians - target_median))
            diff = (np.abs(possible_pixel_medians - self.target_median))

            chosen_idx = possible_idx[np.argsort(diff)[0:self.num_predictor_pixels]]
            
        self.predictor_pixels_locations = np.array([[idx // im_side_length, idx % im_side_length] 
                                                   for idx in chosen_idx])
        loc = self.predictor_pixels_locations.T
        predictor_pixels = np.zeros((self.im_fluxes[0].shape))
        predictor_pixels[loc[0], loc[1]] = 1
        
        self.predictor_pixels_fluxes = self.im_fluxes[:, loc[0], loc[1]]  # shape is (1282, num_predictors)
        self.rescaled_predictor_pixels_fluxes = self.rescaled_im_fluxes[:, loc[0], loc[1]]
        self.predictor_pixels_mask = np.ma.masked_where(predictor_pixels == 0, predictor_pixels)
        
        self.are_predictors_set = True
        
    def train(self, reg):
        if ((self.is_target_set  == False) or (self.is_exclusion_set == False)
           or self.are_predictors_set == False):
            print("You missed a step.")
        
        def objective(coeff, reg):
            model = np.dot(coeff, self.predictor_pixels_fluxes.T)
            chi2 = ((self.target_fluxes - model)/(self.target_errors))**2
            return np.sum(chi2) + reg*np.sum(coeff**2)
            
        init_coeff = np.zeros(self.num_predictor_pixels)
        self.fit = minimize(objective, init_coeff, args=(reg), tol=0.5)
        self.prediction = np.dot(self.fit.x, self.predictor_pixels_fluxes.T)
        print(self.fit.success)
        print(self.fit.message)
        
        self.trained = True
        
    def lsq(self, reg, rescale=True, polynomials=False):
        if ((self.is_target_set  == False) or (self.is_exclusion_set == False)
           or self.are_predictors_set == False):
            print("You missed a step.")
        
        self.regularization = reg
        num_components = self.num_predictor_pixels
        
        if (rescale == False):
            print("Calculating parameters using unscaled values.")
            y = self.target_fluxes
            m = self.predictor_pixels_fluxes  # (num of measurements(1282) , num of predictors (128))
        
        elif (rescale == True):
            y = self.rescaled_target_fluxes
            m = self.rescaled_predictor_pixels_fluxes
            
        if (polynomials == True):
            m = np.hstack((m, self.v_matrix))
            num_components = num_components + self.v_matrix.shape[1]
            
        l = reg*np.identity(num_components)
        a = np.dot(m.T, m) + l
        b = np.dot(m.T, y)
        
        self.lsq_params = np.linalg.solve(a, b)
        self.cpm_params = self.lsq_params[:self.num_predictor_pixels]
        self.poly_params = self.lsq_params[self.num_predictor_pixels:]
        self.cpm_prediction = np.dot(m[:, :self.num_predictor_pixels], self.cpm_params)
        self.poly_prediction = np.dot(m[:, self.num_predictor_pixels:], self.poly_params)
        self.lsq_prediction = np.dot(m, self.lsq_params)
        
        if (rescale == True):
            self.lsq_prediction = np.median(self.target_fluxes)*(self.lsq_prediction + 1)
            if (polynomials == True):
                self.cpm_prediction = np.median(self.target_fluxes)*(self.cpm_prediction + 1)
                self.poly_prediction = np.median(self.target_fluxes)*(self.poly_prediction + 1)
                
        self.trained = True
        
    def get_contributing_pixels(self, number):
        """Return the n-most contributing pixels' locations and a mask to see them"""
        if self.trained == False:
            print("You need to train the model first.")
            
            
        if self.fit == None:
            idx = np.argsort(np.abs(self.cpm_params))[:-(number+1):-1]
        else:
            idx = np.argsort(np.abs(self.fit.x))[:-(number+1):-1]
        
        top_n_loc = self.predictor_pixels_locations[idx]
        loc = top_n_loc.T
        top_n = np.zeros(self.im_fluxes[0].shape)
        top_n[loc[0], loc[1]] = 1
        
        top_n_mask = np.ma.masked_where(top_n == 0, top_n)
        
        return (top_n_loc, top_n_mask)
    
    def entire_image(self, reg):
        self.reg = reg
        self.im_predicted_fluxes = np.empty(self.im_fluxes.shape)
        num_col = self.im_fluxes[0].shape[1]
        idx = np.arange(num_col**2)
        rows = idx // num_col
        cols = idx % num_col
        for (row, col) in zip(rows, cols):
#         for (row, col) in zip(rows[:10], cols[:10]):
            self.set_target(row, col)
            self.set_exclusion(4, method="cross")
            self.set_predictor_pixels(128, method="similar_brightness")
            self.lsq(reg, rescale=True, polynomials=True)
            self.im_predicted_fluxes[:, row, col] = self.cpm_prediction
        self.im_diff = self.im_fluxes - self.im_predicted_fluxes

In [ ]:
def plot_lightcurves(cpm):
    fig, axs = plt.subplots(2, 1, figsize=(18, 12))
    data = cpm.target_fluxes
    model = cpm.lsq_prediction
    res = data - cpm.cpm_prediction
    
    axs[0].plot(cpm.time, data, ".", color="k", label="Data", markersize=6)
    axs[0].plot(cpm.time, model, ".", color="C3", label="Model", markersize=4, alpha=0.6)
    if (cpm.poly_params.shape != (0,)):
        axs[0].plot(cpm.time, cpm.cpm_prediction, ".", color="C1", label="CPM", markersize=4, alpha=0.4)
        axs[0].plot(cpm.time, cpm.poly_prediction, "-", color="C2", label="Poly", markersize=4, alpha=0.7)
    
    axs[1].plot(cpm.time, res, ".-", label="Residual (Data - CPM)", markersize=3)

    for i in range(2):
        axs[i].legend(fontsize=15)
    plt.show()

# def summary_plot(cpm, n):
    
#     top_n_loc, top_n_mask = cpm.get_contributing_pixels(n)
    
#     fig, axs = plt.subplots(1, 3, figsize=(18, 16))
    
# #     first_image = cpm.im_fluxes[0,:,:]
#     first_image = cpm.pixel_medians
#     axs[0].imshow(first_image, origin="lower",
#            vmin=np.percentile(first_image, 10), vmax=np.percentile(first_image, 90))
    
#     axs[1].imshow(first_image, origin="lower",
#            vmin=np.percentile(first_image, 10), vmax=np.percentile(first_image, 90))
#     axs[1].imshow(cpm.excluded_pixels_mask, origin="lower", cmap="Set1", alpha=0.5)
#     axs[1].imshow(cpm.target_pixel_mask, origin="lower", cmap="binary", alpha=1.0)
#     axs[1].imshow(cpm.predictor_pixels_mask, origin="lower", cmap="binary_r", alpha=0.9)
    
#     axs[2].imshow(first_image, origin="lower",
#            vmin=np.percentile(first_image, 10), vmax=np.percentile(first_image, 90))
#     axs[2].imshow(cpm.excluded_pixels_mask, origin="lower", cmap="Set1", alpha=0.5)
#     axs[2].imshow(cpm.target_pixel_mask, origin="lower", cmap="binary", alpha=1.0)
#     axs[2].imshow(cpm.predictor_pixels_mask, origin="lower", cmap="binary_r", alpha=0.9)
#     axs[2].imshow(top_n_mask, origin="lower", cmap="Set1")
    
#     plt.show()
    
#     fig, axs = plt.subplots(2, 1, figsize=(18, 15))
#     data = cpm.target_fluxes
#     model = cpm.lsq_prediction
#     res = data - model
    
#     axs[0].plot(cpm.time, data, ".", color="k", label="Data", markersize=3)
#     axs[0].plot(cpm.time, model, ".", color="C3", label="Model", markersize=3, alpha=0.4)
#     axs[1].plot(cpm.time, res, ".-", label="Residual (Data - Model)", markersize=7)

#     for i in range(2):
#         axs[i].legend(fontsize=15)
    
#     plt.show()
    
def summary_one_plot(cpm, n, save=False):
    top_n_loc, top_n_mask = cpm.get_contributing_pixels(n)
    
    plt.figure(figsize=(18, 14))
    
    ax1 = plt.subplot2grid((4, 3), (0, 0), rowspan=2)
    ax2 = plt.subplot2grid((4, 3), (0, 1), rowspan=2)
    ax3 = plt.subplot2grid((4, 3), (0, 2), rowspan=2)
    
    ax4 = plt.subplot2grid((4, 3), (2, 0), colspan=3)
    ax5 = plt.subplot2grid((4, 3), (3, 0), colspan=3)
    
    first_image = cpm.pixel_medians

#     first_image = cpm.im_fluxes[0,:,:]
    ax1.imshow(first_image, origin="lower",
           vmin=np.percentile(first_image, 10), vmax=np.percentile(first_image, 90))
    
    ax2.imshow(first_image, origin="lower",
           vmin=np.percentile(first_image, 10), vmax=np.percentile(first_image, 90))
    ax2.imshow(cpm.excluded_pixels_mask, origin="lower", cmap="Set1", alpha=0.5)
    ax2.imshow(cpm.target_pixel_mask, origin="lower", cmap="binary", alpha=1.0)
    ax2.imshow(cpm.predictor_pixels_mask, origin="lower", cmap="binary_r", alpha=0.9)
    
    ax3.imshow(first_image, origin="lower",
           vmin=np.percentile(first_image, 10), vmax=np.percentile(first_image, 90))
    ax3.imshow(cpm.excluded_pixels_mask, origin="lower", cmap="Set1", alpha=0.5)
    ax3.imshow(cpm.target_pixel_mask, origin="lower", cmap="binary", alpha=1.0)
    ax3.imshow(cpm.predictor_pixels_mask, origin="lower", cmap="binary_r", alpha=0.9)
    ax3.imshow(top_n_mask, origin="lower", cmap="Set1")
    
    data = cpm.target_fluxes
    model = cpm.lsq_prediction
#     res = data - model
    res = data - cpm.cpm_prediction
    
    ax4.plot(cpm.time, data, ".", color="k", label="Data", markersize=4)
    ax4.plot(cpm.time, model, ".", color="C3", label="Model", markersize=4, alpha=0.4)
    if (cpm.poly_params.shape != (0,)):
        ax4.plot(cpm.time, cpm.cpm_prediction, ".", color="C1", label="CPM", markersize=3, alpha=0.4)
        ax4.plot(cpm.time, cpm.poly_prediction, ".", color="C2", label="Poly", markersize=3, alpha=0.4)
    
#     ax5.plot(cpm.time, res, ".-", label="Residual (Data - Model)", markersize=7)
    ax5.plot(cpm.time, res, ".-", label="Residual (Data - CPM)", markersize=7)
    
    plt.suptitle("N={} Predictor Pixels, Method: {}, L2Reg={}".format(cpm.num_predictor_pixels,
                cpm.method_predictor_pixels, "{:.0e}".format(cpm.regularization)), y=0.89, fontsize=15)
    
    ax1.set_title("Cmap set to 10%, 90% values of image")
    ax2.set_title("Target (White), Excluded (Red Shade), Predictors (Black)")
    ax3.set_title("Top N={} contributing pixels to prediction (Red)".format(n))
    
    ax4.set_title("Lightcurves of Target Pixel {}".format((cpm.target_row, cpm.target_col)))
    ax4.set_ylabel("Flux [e-/s]")
    ax4.legend(fontsize=12)
    
    ax5.set_xlabel("Time (BJD-2457000) [Day]", fontsize=12)
    ax5.set_ylabel("Flux [e-/s]")
    ax5.legend(fontsize=12)
    
    if (save == True):
        plt.savefig("cpm_target_{}_reg_{}_filename_{}.png".format((cpm.target_row,cpm.target_col),
                    "{:.0e}".format(cpm.regularization), cpm.file_name), dpi=200)

def aperture_photometry(row, col, size, diff):
    aperture = diff[:, max(0, row-size):min(row+size+1, diff.shape[1]), 
                    max(0, col-size):min(col+size+1, diff.shape[1])]
    aperture_lc = np.sum(aperture, axis=(1, 2))
    return aperture, aperture_lc

In [ ]:
# fits_file = "tpf/tess-s0001-3-3_36.820737_-82.019560_64x64_astrocut.fits"
# fits_file = "tpf/tess-s0011-4-1_63.278529_-72.820643_64x64_astrocut.fits"
# fits_file = "tpf/tess-s0011-4-4_64.525833_-63.615669_64x64_astrocut.fits"
sector_1 = "tpf/tess-s0001-4-1_64.525833_-63.615669_64x64_astrocut.fits"

In [ ]:
a = CPM(sector_1, remove_bad=True)

In [ ]:
a.entire_image(0.5)

In [ ]:
a.set_target(32, 32)
# cpm.set_target(8, 40)

a.set_exclusion(4, method="cross")
a.set_predictor_pixels(128, method="similar_brightness", seed=2)

a.lsq(0.5, rescale=True, polynomials=True)

summary_one_plot(a, 10)
plot_lightcurves(a)

In [ ]:
sector_2 = "tpf/tess-s0002-4-1_64.525833_-63.615669_64x64_astrocut.fits"
b = CPM(sector_2, remove_bad=True)
b.entire_image(0.5)
# # b.set_target(15, 35)
b.set_target(32, 32)

b.set_exclusion(4, method="cross")
b.set_predictor_pixels(128, method="similar_brightness", seed=2)

b.lsq(0.5, rescale=True, polynomials=True)

# summary_one_plot(b, 3)
plot_lightcurves(b)

In [ ]:
plt.figure(figsize=(18, 10))
s1, s1_lc = aperture_photometry(32, 32, 1, a.im_diff)
s2, s2_lc = aperture_photometry(32, 32, 1, b.im_diff)
# s3 = aperture_photometry(32, 32, 3, b.im_diff)
# s4 = aperture_photometry(32, 32, 4, b.im_diff)
# s5 = aperture_photometry(32, 32, 5, b.im_diff)


plt.plot(a.time, a.im_diff[:, 32, 32], ".", label="Sector 1 Single Pixel")
plt.plot(b.time, b.im_diff[:, 32, 32] + a.im_diff[-1, 32, 32], ".", label="Sector 2 Single Pixel")
plt.plot(a.time, s1_lc, ".", label="Sector 1 3x3")
plt.plot(b.time, s2_lc + s1_lc[-1], ".", label="Sector 2 3x3")
# plt.plot(b.time, s3, label="7x7")
# plt.plot(b.time, s4, label="9x9")
# plt.plot(b.time, s5, label="11x11")
plt.legend()

In [ ]:
apt, lc = aperture_photometry(32, 32, 1, a.im_diff)
plt.figure(figsize=(18, 10))

apt_one = apt.reshape(apt.shape[0], apt.shape[1]**2)
for i in range(apt_one.shape[1]):
    plt.plot(a.time, apt_one[:, i])
plt.plot(a.time, a.im_diff[:, 32, 32], ".",alpha=1.0)

In [ ]:
# diff = a.im_fluxes - a.im_predicted_fluxes
aperture_lc = aperture_photometry(32, 32, b.im_diff)
plt.figure(figsize=(14, 10))
plt.plot(b.time, aperture_lc, ".")

In [ ]:
plt.plot(a.time, a.im_diff[:, 32, 32])

In [ ]:
diff = a.im_fluxes - a.im_predicted_fluxes

from IPython.display import HTML
import matplotlib.animation as animation

fig = plt.figure(figsize=(10, 10))

ims = []
for i in range(0, diff.shape[0], 5):
    im = plt.imshow(diff[i], origin="lower", animated=True,
                   vmin=np.percentile(diff, 1), vmax=np.percentile(diff, 99));
    ims.append([im]);
fig.colorbar(im)
    
ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True,
                                repeat_delay=1000);

HTML(ani.to_jshtml())

In [ ]:
a.set_target(32, 32)
a.set_exclusion(4, method="cross")
a.set_predictor_pixels(128, method="similar_brightness", seed=2)

reg = 1.0
a.lsq(reg, rescale=True)
summary_one_plot(a, 10, save=True)

plt.figure(figsize=(15, 12))
plt.plot(a.lsq_params, ".", markersize=10)
plt.xlabel("Coefficient", fontsize=15)
plt.ylabel("Amplitude of Coefficient", fontsize=15)
plt.axhline(0.0, color="k")

In [ ]:
diff = a.im_fluxes - a.im_predicted_fluxes
plt.figure(figsize=(15, 10))
plt.plot(a.time, aperture_photometry(32, 32, diff), ".")

In [ ]:
data = a.target_fluxes
err = a.target_errors

regs = np.linspace(0, 1, 1000)
res_array = []
for reg in regs:
    a.lsq(reg)
    prediction = a.lsq_prediction
    diff = data - prediction
    sum_res = np.sum((diff/err)**2)
    res_array.append(sum_res)

plt.figure(figsize=(14, 8))
plt.plot(regs, res_array, "o-")
# labs = ["{:.0e}".format(reg) for reg in regs]
# print(labs)
# plt.xticks(np.arange(9), labels=labs);
plt.xlabel("Regularization Value", fontsize=15)
plt.ylabel("Sum of Squared Residuals", fontsize=15)
plt.xscale("log")

In [ ]:
fits_file = "tpf/tess-s0002-4-1_64.525833_-63.615669_64x64_astrocut.fits"

In [ ]:
cpm = CPM(fits_file, remove_bad=True)
cpm.set_target(32, 32)

cpm.set_exclusion(4, method="cross")
cpm.set_predictor_pixels(128, method="similar_brightness", seed=2)

reg = 0.1
cpm.lsq(reg, rescale=True, polynomials=True)

summary_one_plot(cpm, 10)

plt.figure(figsize=(15, 12))
plt.plot(np.arange(cpm.num_predictor_pixels), cpm.cpm_params, ".", markersize=10)
plt.plot(np.arange(cpm.num_predictor_pixels, cpm.lsq_params.shape[0]), cpm.poly_params, ".", markersize=10)

plt.xlabel("Coefficient", fontsize=15)
plt.ylabel("Amplitude of Coefficient", fontsize=15)
plt.axhline(0.0, color="k")

In [ ]:
cpm.entire_image(reg)
diff = cpm.im_fluxes - cpm.im_predicted_fluxes
plt.figure(figsize=(15, 10))
plt.plot(cpm.time, aperture_photometry(32, 32, diff), ".")

In [ ]:
cpm.set_target(32, 32)

cpm.set_exclusion(4, method="cross")
cpm.set_predictor_pixels(128, method="similar_brightness", seed=2)

data = cpm.target_fluxes
err = cpm.target_errors

regs = np.linspace(0, 1, 1000)
res_array = []
for reg in regs:
    cpm.lsq(reg)
    prediction = cpm.lsq_prediction
    diff = data - prediction
    sum_res = np.sum((diff/err)**2)
    res_array.append(sum_res)

plt.figure(figsize=(14, 8))
plt.plot(regs, res_array, "o-")
# labs = ["{:.0e}".format(reg) for reg in regs]
# print(labs)
# plt.xticks(np.arange(9), labels=labs);
plt.xlabel("Regularization Value", fontsize=15)
plt.ylabel("Sum of Squared Residuals", fontsize=15)
plt.xscale("log")


In [ ]:
from IPython.display import HTML
import matplotlib.animation as animation

fig = plt.figure(figsize=(10, 10))

ims = []
for i in range(0, diff.shape[0], 5):
    im = plt.imshow(diff[i], origin="lower", animated=True,
                   vmin=np.percentile(diff, 1), vmax=np.percentile(diff, 99));
    ims.append([im]);
fig.colorbar(im)
    
ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True,
                                repeat_delay=1000);

HTML(ani.to_jshtml())

In [ ]:
data = cpm.target_fluxes
err = cpm.target_errors

regs = np.linspace(0, 0.1, 100)
res_array = []
for reg in regs:
    cpm.lsq(reg)
    prediction = cpm.lsq_prediction
    diff = data - prediction
    sum_res = np.sum((diff/err)**2)
    res_array.append(sum_res)

plt.figure(figsize=(14, 8))
plt.plot(regs, res_array, "o-")
# labs = ["{:.0e}".format(reg) for reg in regs]
# print(labs)
# plt.xticks(np.arange(9), labels=labs);
plt.xlabel("Regularization Value", fontsize=15)
plt.ylabel("Sum of Residuals Squared", fontsize=15)

In [ ]:
plt.figure(figsize=(14, 14))
plt.imshow(c.im_fluxes[0], origin="lower")
plt.colorbar()

In [ ]:
c = CPM(fits_file)
c.set_target(32, 32)
c.set_exclusion(5, method="cross")
c.set_predictor_pixels(128, method="similar_brightness")

data = c.target_fluxes
err = c.target_errors

regs = np.hstack((np.array(0), 10**np.arange(9)))
res_array = []
for reg in regs:
    c.lsq(reg)
    prediction = c.lsq_prediction
    diff = data - prediction
    sum_res = np.sum((diff/err)**2)
    res_array.append(sum_res)

plt.figure(figsize=(14, 8))
plt.plot(res_array, "o-")
labs = ["{:.0e}".format(reg) for reg in regs]
# print(labs)
plt.xticks(np.arange(9), labels=labs);
plt.xlabel("Regularization Value", fontsize=15)
plt.ylabel("Sum of Residuals Squared", fontsize=15)

In [ ]:
fits_file3 = "tpf/tess-s0001-3-3_15.000000_-82.000000_64x64_astrocut.fits"
c = CPM(fits_file3)

c.set_target(11, 39)

c.set_exclusion(3, method="cross")
# c.set_predictor_pixels(128, method="random", seed=5)
c.set_predictor_pixels(128, method="similar_brightness", seed=5)

c.lsq(1e4)
summary_one_plot(c, 15)

data = c.target_fluxes
err = c.target_errors
regs = np.hstack((np.array(0), 10**np.arange(5)))
res_array = []
for reg in regs:
    c.lsq(reg)
    prediction = c.lsq_prediction
    diff = data - prediction
    sum_res = np.sum((diff/err)**2)
    res_array.append(sum_res)

plt.figure(figsize=(14, 8))
plt.plot(res_array, "o-")
labs = ["{:.0e}".format(reg) for reg in regs]
# print(labs)
plt.xticks(np.arange(5), labels=labs);
plt.xlabel("Regularization Value", fontsize=15)
plt.ylabel("Sum of Residuals Squared", fontsize=15)

In [ ]:
c.lsq(1e4)
summary_one_plot(c, 15)

In [ ]:
def cpm_over_image(fits_file):
    cpm = CPM(fits_file, remove_bad=True)
    im_predicted_fluxes = np.empty(cpm.im_fluxes.shape)
    num_col = cpm.im_fluxes[0].shape[1]
    idx = np.arange(num_col**2)
    rows = idx // num_col
    cols = idx % num_col
    for (row, col) in zip(rows, cols):
        cpm.set_target(row, col)
        cpm.set_exclusion(4, method="cross")
        cpm.set_predictor_pixels(128, method="similar_brightness")
        cpm.lsq(10.0, rescale=True)
        im_predicted_fluxes[:, row, col] = cpm.lsq_prediction
    return cpm, im_predicted_fluxes

In [ ]:
fits_file = "tpf/tess-s0001-4-1_64.525833_-63.615669_64x64_astrocut.fits"

In [ ]:
%%timeit
row = idx[0] // 64
col = idx[0] // 64

In [ ]:
cpm, im_predicted_fluxes = cpm_over_image(fits_file)

In [ ]:
c = CPM(fits_file, remove_bad=True)
c.set_target(32, 32)
c.set_exclusion(4, method="cross")
c.set_predictor_pixels(128, method="similar_brightness")
c.lsq(1, rescale=True)
summary_one_plot(c, 15)

In [ ]:
data = c.target_fluxes
err = c.target_errors

regs = np.hstack((np.array(0), 10**np.arange(5)))
res_array = []
for reg in regs:
    c.lsq(reg)
    prediction = c.lsq_prediction
    diff = data - prediction
    sum_sq_res = np.sum((diff/err)**2)
    res_array.append(sum_sq_res)

plt.figure(figsize=(14, 8))
plt.plot(res_array, "o-")
labs = ["{:.0e}".format(reg) for reg in regs]
plt.xticks(np.arange(9), labels=labs);
plt.xlabel("Regularization Value", fontsize=15)
plt.ylabel("Sum of Squared Residuals", fontsize=15)

In [ ]:
diff = cpm.im_fluxes - im_predicted_fluxes
# print(np.argmax(np.max(diff, axis=0).reshape(4096)))
# print(1010 // 64, 1010 % 64)

from IPython.display import HTML
import matplotlib.animation as animation

fig = plt.figure(figsize=(12, 12))

ims = []
for i in range(0, diff.shape[0], 10):
    im = plt.imshow(diff[i], origin="lower", animated=True,
                   vmin=np.percentile(diff, 1), vmax=np.percentile(diff, 99));
    ims.append([im]);
fig.colorbar(im)
    
ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True,
                                repeat_delay=1000);

HTML(ani.to_jshtml())

In [ ]:
a = CPM(fits_file)
# cpm.set_target(2, 32)  # random empty spot
a.set_target(32, 32)  # bottom right 

a.set_exclusion(4, method="cross")
a.set_predictor_pixels(128, method="similar_brightness", seed=2)

reg = 0.1
# cross_bottom_right.train(reg)
a.lsq(reg, rescale=True, polynomials=True)

summary_one_plot(a, 10)

In [ ]:
%%timeit
reg = 1e4
l = reg*np.identity(128)

In [ ]:
%%timeit
np.median(a.im_fluxes, axis=0)

In [ ]:
%%timeit
a = CPM(fits_file)

In [ ]:
%%timeit 
a.set_target(32, 32)

In [ ]:
%%timeit
a.set_exclusion(8, method="row_exclude")

In [ ]:
%%timeit
a.set_predictor_pixels(128, method="similar_brightness")

In [ ]:
%%timeit 
a.lsq(0.1, rescale=True, polynomials=True)

In [ ]:
a.entire_image(0.1)

In [ ]:
fits_sector_11 = "tpf/tess-s0011-4-1_63.278529_-72.820643_64x64_astrocut.fits"
c = CPM(fits_sector_11)
c.set_target(32, 32)
c.set_exclusion(5, method="cross")
c.set_predictor_pixels(64)
c.lsq(0.1)

summary_one_plot(c, 32)

In [ ]:
data = c.target_fluxes
err = c.target_errors

regs = np.hstack((np.array(0), 10**np.arange(9)))
regs = np.linspace(0, 0.1, 10)
res_array = []
for reg in regs:
    c.lsq(reg)
    prediction = c.lsq_prediction
    diff = data - prediction
    sum_sq_res = np.sum((diff/err)**2)
    res_array.append(sum_sq_res)

plt.figure(figsize=(14, 8))
plt.plot(regs, res_array, "o-")
# labs = ["{:.0e}".format(reg) for reg in regs]
# plt.xticks(np.arange(9), labels=labs);
plt.xlabel("Regularization Value", fontsize=15)
plt.ylabel("Sum of Squared Residuals", fontsize=15)

In [ ]:
c, im_predicted_fluxes = cpm_over_image(fits_sector_11)

In [ ]:
diff = c.im_fluxes - im_predicted_fluxes
# print(np.argmax(np.max(diff, axis=0).reshape(4096)))
# print(1010 // 64, 1010 % 64)

from IPython.display import HTML
import matplotlib.animation as animation

fig = plt.figure(figsize=(10, 10))

ims = []
for i in range(0, diff.shape[0], 5):
    im = plt.imshow(diff[i], origin="lower", animated=True,
                   vmin=np.percentile(diff, 2), vmax=np.percentile(diff, 98));
    ims.append([im]);
fig.colorbar(im)
    
ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True,
                                repeat_delay=1000);

HTML(ani.to_jshtml())

In [ ]:
np.iszero?